In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## படி 1: கட்டமைக்கப்பட்ட வெளியீடுகளுக்கான Pydantic மாதிரிகள் வரையறுக்கவும்

இந்த மாதிரிகள் முகவர்கள் வழங்கும் **திட்டத்தை** வரையறுக்கின்றன. Pydantic உடன் `response_format` பயன்படுத்துவதன் மூலம் নিশ্চিতிக்கப்படுகிறது:
- ✅ வகை-பாதுகாப்பான தரவு மீட்டெடு
- ✅ தானாகச் சோதனை
- ✅ சுதந்திர உரை பதிலுகளில் பார்்சிங் பிழைகள் இல்லை
- ✅ புலங்களைப் பின்பற்றி எளிய நிபந்தனையுடன் வழிமாற்றம்


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## படி 2: ஹோட்டல் முன்பதிவு கருவியை உருவாக்கவும்

இந்நூல் **availability_agent** அறைகள் கிடைக்கும் என்பதைச் சரிபார்க்க அழைக்கும் கருவி ஆகும். இதற்காக `@ai_function` டெக்கரேட்டரைப் பயன்படுத்துகிறோம்:
- ஒரு Python செயல்பாட்டை AI-அழைக்கக்கூடிய கருவியாக மாற்ற
- LLM க்கான JSON கட்டமைப்பை தானாக உருவாக்க
- அளவுரு சரிபார்ப்பை கையாள
- முகவரிகளால் தானாக அழைப்பை இயக்கு

இந்த காட்சிக்காக:
- **ஸ்டாக்ஹோல்ம், சீட்டிள், டோக்கியோ, லண்டன், அம்ஸ்டர்டாம்** → அறைகள் கிடைக்கும் ✅
- **மற்ற அனைத்து நகரங்களும்** → அறைகள் கிடையாது ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## படி 3: வழித்துணைக்கு நிலைமைக் காரியங்களை வரையறு

இந்தக் காரியங்கள் агентன் பதிலை ஆய்வு செய்து பணிப் பாசையில் எது எடுத்து செல்ல வேண்டும் என்பதைக் கண்டறிகின்றன.

**முக்கிய வடிவமைப்பு:**
1. செய்தி `AgentExecutorResponse` ஆக உள்ளதா என சரிபார்
2. கட்டமைக்கப்பட்ட வெளியீடை (Pydantic மாதிரி) பகுத்தறி
3. வழித்துணையை கட்டுப்படுத்த `True` அல்லது `False` 반환ி

இந்த நிலைகள் **கடைகள்** மீது மதிப்பாய்வு செய்யப்படும் அடுத்து எந்த செயல்படுத்துனரை அழைக்க வேண்டும் என்பதனை தீர்மானிக்க.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## படி 4: தனிப்பயன் காட்சிப்படுத்தி இயக்குநரை உருவாக்குங்கள்

இயக்குனர்கள் மாற்றங்கள் அல்லது பக்க விளைவுகளை செயல்படுத்தும் வேலைப்பாட்டுக் கூறுகள் ஆகும். இறுதி முடிவைக் காட்சிப்படுத்த `@executor` அலங்காரியை பயன்படுத்தி தனிப்பயன் இயக்குநரை நாம் உருவாக்குகிறோம்.

**முக்கிய கருத்துகள்:**
- `@executor(id="...")` - ஒரு செயல்பாட்டை வேலைப்பாட்டு இயக்குநராக பதிவு செய்கிறது
- `WorkflowContext[Never, str]` - உள்ளீடு/வெளியீட்டிற்கான வகை குறிப்புகள்
- `ctx.yield_output(...)` - இறுதி வேலைப்பாட்டு முடிவை வழங்குகிறது


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Step 5: சூழல் மாறிலிகள் ஏற்றவும்

LLM கிளையண்டை 구성ிக்கவும். இந்த உதாரணம் பின்வரும் மாதிரிகளுடன் வேலை செய்கிறது:
- **GitHub மாதிரிகள்** (GitHub டோக்கன் கொண்ட Free tier)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## படி 6: கட்டமைக்கப்பட்ட வெளியீடுகளுடன் AI முகவரிகளை உருவாக்குதல்

நாம் **மூன்று சிறப்பு முகவரிகளை** உருவாக்குகிறோம், ஒவ்வொன்றும் `AgentExecutor` இல் சுவரம் படுத்தப்பட்டுள்ளது:

1. **availability_agent** - கருவியைப் பயன்படுத்தி ஹோட்டல் கிடைக்கும் தரவரிசையைச் சரிபார்க்கிறது  
2. **alternative_agent** - மாற்று நகரங்களை பரிந்துரைக்கிறது (இடங்கள் இல்லாதபோது)  
3. **booking_agent** - முன்பதிவை ஊக்கப்படுத்துகிறது (இடங்கள் கிடைக்கும் போது)  

**முக்கிய அம்சங்கள்:**  
- `tools=[hotel_booking]` - முகவரிக்கு கருவியை வழங்குகிறது  
- `response_format=PydanticModel` - கட்டமைக்கப்பட்ட JSON வெளியீட்டை வலியுறுத்துகிறது  
- `AgentExecutor(..., id="...")` - வேலைப்ப流க்கதிற்கு முகவரியை சுவரம் படுத்துகிறது


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## படி 7: நிபந்தனை விளிம்புகளுடன் வேலைபார்வையை கட்டமைக்கவும்

இப்போது நாங்கள் `WorkflowBuilder` ஐ நிபந்தனை வழிசெலுத்தலை கொண்டு வரைபடத்தை கட்டமைப்பதற்கு பயன்படுத்துகிறோம்:

**வேலைபார்வை அமைப்பு:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**முக்கிய முறைகள்:**
- `.set_start_executor(...)` - நுழைவு இடத்தை அமைக்கிறது
- `.add_edge(from, to, condition=...)` - நிபந்தனை விளிம்பு சேர்க்கிறது
- `.build()` - வேலைபார்வையை இறுதி கட்டம் செய்கிறது


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## படி 8: சோதனை வழக்கு 1 - கிடைப்பில்லாத நகரம் (பேரிஸ்)

நமது சிமுலேஷனில் அறைகள் இல்லாத பேரிசில் ஹோட்டல்களை கோருவதன் மூலம் **கிடைப்பில்லை** பாதையை சோதிக்கலாம்.


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## படி 9: சோதனை வழக்கு 2 ஐ இயக்கவும் - நகரம் கிடைக்கும் நிலை உடன் (ஸ்டாக்ஹோம்)

கிடைக்கும் பாதையை சோதிப்போம், ஸ்டாக்ஹோம் நகரத்தில் உள்ள ஹோட்டல்களை கேட்கும்போது (எமது சிம்யுலேஷனில் அறைகள் உள்ளன).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## முக்கியமான எடுத்துக்காட்டுகள் மற்றும் அடுத்து செய்ய வேண்டியவை

### ✅ நீங்கள் கற்றுக்கொண்டதை:

1. **WorkflowBuilder மாதிரி**
   -  நுழைவுகருவாயை வரையறுக்க `.set_start_executor()` பயன்படுத்தவும்
   -  நிபந்தனை அடிப்படையிலான வழிச்செலுத்தலுக்கு `.add_edge(from, to, condition=...)` பயன்படுத்தவும்
   - வேலைவழிக்கலை முடிக்க `.build()` அழைக்கவும்

2. **நிபந்தனை அடிப்படையிலான வழிச்செலுத்தல்**
   -  நிபந்தனை செயல்பாடுகள் `AgentExecutorResponse` ஐ பரிசோதிக்கின்றன
   -  வழிசெலுத்தல் முடிவுகளுக்கு அமைந்த கட்டமைக்கப்பட்ட வெளியீடுகளை பாகுபாடு செய்கின்றன
   -  ஒரு வழிச்சிமையை செயல்படுத்த `True` ஐத் திருப்பவும், தவிர்க்க `False` ஐத் திருப்பவும்

3. **கருவிகள் ஒருங்கிணைப்பு**
   -  பைதான் செயல்பாடுகளை AI கருவிகளாக மாற்ற `@ai_function` ஐ பயன்படுத்தவும்
   -  முகவர்கள் தேவையான போது தானாக கருவிகளை அழைக்கின்றனர்
   -  கருவிகள் முகவர்கள் பாகுபாடு செய்யக்கூடிய JSON ஐ திருப்புகின்றன

4. **கட்டமைக்கப்பட்ட வெளியீடுகள்**
   -  தரவுகளை பாதுகாப்புடன் எடுக்க Pydantic மாதிரிகள் பயன்படுத்தவும்
   -  முகவர்கள் உருவாக்கும்போது `response_format=MyModel` என்றும் அமைக்கவும்
   -  பதில்களை `Model.model_validate_json()` பயன்படுத்தி பாகுபாடு செய்யவும்

5. **சொந்த செயல்படுத்திகள்**
   -  வேலைவழிக்கல் கூறுகளை உருவாக்க `@executor(id="...")` பயன்படுத்தவும்
   -  செயல்படுத்திகள் தரவை மாற்றவோ அல்லது பக்க விளைவுகளை செய்யவோ முடியும்
   -  வேலைவழிக்கலின் முடிவுகளை உருவாக்க `ctx.yield_output()` பயன்படுத்தவும்

### 🚀 உண்மை உலக பயன்பாடுகள்:

- **பயணமாக்கல் முன்பதிவு**: கிடைப்பதை சரிபார், மாற்று முன்முயற்சிகள் பரிந்துரை செய், விருப்பங்களிடையே ஒப்பிடு
- **வாடிக்கையாளர் சேவை**: சிக்கல் வகை, உணர்ச்சி, முக்கியத்துவம் அடிப்படையில் வழித்தொடர்
- **மின்வணிகம்**: பராமரிப்பு சரிபார், மாற்றுகளை பரிந்துரை செய், ஆர்டர்களை செயலாக்கு
- **உள்ளடக்க மேலாண்மை**: முரண்பாட்டுக் கோலங்களுக்கு, பயனர் இடப்பட்ட குறி அடிப்படையில் வழிசெலுத்தல்
- **அங்கீகார வேலைவழிகள்**: தொகை, பயனர் பங்கு, அபாயம் நிலை அடிப்படையில் வழிசெலுத்தல்
- **பல கட்ட பணிகள்**: தர தரம், முழுமை அடிப்படையில் வழிசெலுத்தல்

### 📚 அடுத்து செய்ய வேண்டியவை:

- மேலும் சிக்கலான நிபந்தனைகளை (பல நிபுணத்துவங்கள்) சேர்க்கவும்
- வேலைவழிக்கல் நிலை நிர்வாகத்துடன் சூழல் (loops) செயல்படுத்தவும்
- மீண்டும் பயன்படுத்தக்கூடிய கூறுகளுக்கான துணை-வேலைவழிகளை சேர்க்கவும்
- உண்மையான API களுடன் ஒருங்கிணைக்கவும் (உதா. ஹோட்டல் முன்பதிவு, பராமரிப்பு அமைப்புகள்)
- பிழை மேலாண்மை மற்றும் வழிமாற்று பாதைகளை சேர்க்கவும்
- உட்பிரவேசப்பட்ட காட்சிப்படுத்தல் கருவிகளுடன் வேலைவழிகளை காட்சிமயமாக்கவும்


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**விடுபடைப்பு**:
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக போராடினாலும், தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்க வாய்ப்புள்ளது என்பதை கவனிக்கவும். அசல் ஆவணம் அதன் இயல்புநிலையான மொழியில் அதிகாரபூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கிய தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்புநர் மூலம் மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பின் பயன்படுத்தலால் ஏற்படும் எந்தப் புரிதல்த் தவறுகளுக்கும் நாங்கள் பொறுப்பல்லாம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
